#### Importing libraries

In [ ]:
!pip install mahaNLP==0.5
from mahaNLP.datasets import *
from mahaNLP.tokenizer import Tokenize
import pandas as pd

#### Load the dataset to be augmented 

In [ ]:
data_train = pd.read_csv('/kaggle/input/d/ishaabhijitjoshi/mahasent-part1/mahasent_train_new_part1.csv')#dataset['tweets-train']

In [ ]:
data_train.shape

#### Preprocessing

In [ ]:
## Clean dataset

import re
def remove_non_marathi(i):
    i= re.sub(r"http\S+", "", i)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#\S+", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)
    return i
  

In [ ]:
data_train.head()

In [ ]:
from mahaNLP.modelRepo import MaskFillModel, GPTModel, HateModel, NERModel, SentimentModel, SimilarityModel

In [ ]:
modelMaskFill = MaskFillModel()

#### Generate the positions for the random words to be replaced 

In [ ]:

positions = []

from random import *
for i in range(0,data_train.shape[0]):
    text = data_train.marathi_text.iloc[i].split()
    rand_num = []
    count = 0
    while count<=(40*len(text))//100:
        num = randint(0,len(text)-1)
        if(num not in rand_num):
            rand_num.append(num)
            count = count + 1
    rand_num.sort()
    positions.append(rand_num)
    print(i)


In [ ]:
model = Tokenize()

#### Sequential augmentation

In [ ]:
# For sequential

import re


import tensorflow as tf

with tf.device('/gpu:0'):
    sentences = []
    inc = 0
    for i in range(0,data_train.shape[0]):    
        #text = data_train.tweet.iloc[i].split()
        text = model.wordTokenize(data_train.tweet.iloc[i])
        count = 0
        tokens = []
        for j in positions[i]:
            if j<len(text):
                temp = text[j]
                text[j] = "[MASK]"
                var = modelMaskFill.predictMask(" ".join(text)).token_str
                if(var.iloc[0]==temp):
                    text[j] = var.iloc[1]
                else:
                    text[j] = var.iloc[0]
            #print(temp, " => ",text[j]) ## added
        final = " ".join(text)
        final = re.sub(" ##",'',final)
        sentences.append(final)
        print(i," => ",len(final))
        
        if i%1000==0 and i!=0:
            df_n = pd.DataFrame(
            {'original':data_train.tweet[inc:i],
             'sequential': sentences[0:1000],
             'positions':positions[inc:i]
            })      

            df_n.to_csv('sequential_sentences_'+str(i)+'.csv')
            inc += 1000
            sentences = []

        if i== data_train.shape[0]-1:
            df_n = pd.DataFrame(
            {'original':data_train.tweet[12000:i],
             'sequential': sentences[0:i],
             'positions':positions[12000:i]
            })      

            df_n.to_csv('sequential_sentences_'+str(i)+'.csv')
        
      
        
    

#### Parallel augmentation

In [ ]:

import tensorflow as tf

with tf.device('/gpu:0'):
    sentences2 = []
    inc = 0
    for i in range(0,data_train.shape[0]):    
        #text = data_train.tweet.iloc[i].split()
        text = model.wordTokenize(data_train.marathi_text.iloc[i])
        count = 0
        tokens = []
        for j in positions[i]:
            if j<len(text):
                temp = text[j]
                text[j] = "[MASK]"
                var = modelMaskFill.predictMask(" ".join(text)).token_str
                text[j] = temp
                if var.iloc[0] == temp:
                    tokens.append(var.iloc[1])
                else:
                    tokens.append(var.iloc[0])
        c = 0
        for k in positions[i]:
            if k<len(text):
                text[k] = tokens[c]
                c += 1
        final = " ".join(text)
        final = re.sub(" ##",'',final)
        sentences2.append(final)
        print(i," => ",len(final))
        if i%1000==0 and i!=0:
            df_n = pd.DataFrame(
            {'original':data_train.marathi_text[inc:i],
             'parallel': sentences2[inc:i],
             'positions':positions[inc:i],
             'labels': data_train.three_point[inc:i]
            })      

            df_n.to_csv('parallel_sentences_'+str(i)+'.csv')
            inc += 1000
        if i== data_train.shape[0]-1:
            df_n = pd.DataFrame(
            {'original':data_train.marathi_text[inc:i],
             'sequential': sentences2[inc:i],
             'positions':positions[inc:i],
             'labels': data_train.three_point[inc:i]
            })      

            df_n.to_csv('parallel_sentences_'+str(i)+'.csv')
